# Get Faculty Names from NetID
This notebook login to outlook (cmsetech@msu.edu), search for the NetID and get the corresponding names of the faculty member

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
import re
import numpy as np

from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from getpass import getpass
from bs4 import BeautifulSoup

In [ ]:
emails = pd.read_csv("CMSE_Email_lists.csv")
emails

cmse_facs = [emails['cmse_reg_fac'], emails['cmse_voting_fac'], emails['cmse_fac']]
fac_series = pd.concat(cmse_facs, ignore_index=True).drop_duplicates()
df = pd.DataFrame(fac_series, columns=['CMSE Faculty NetID'])

df['CMSE Faculty NetID'] = df['CMSE Faculty NetID'].apply(lambda txt: None if pd.isna(txt) or "cmse" in txt else txt.split('@')[0])
df.dropna(inplace=True)
df = df.reset_index(drop=True)
df.replace('markiwen', 'iwenmark', inplace=True) # Prof. Iwen new actual netID
df

In [ ]:
driver = webdriver.Firefox()
time.sleep(5)

In [ ]:
url = "https://outlook.office.com/mail/"
driver.get(url)
time.sleep(5)

element = driver.find_element(By.ID, 'i0116') 
element.send_keys('cmsetech@msu.edu')

element = driver.find_element(By.ID, 'idSIButton9')
element.click()
time.sleep(5)

In [ ]:
element = driver.find_element(By.ID, 'input36') # Find the password input
print("Please enter the CMSETech password: ")
password = getpass()
element.send_keys(password)

element = driver.find_element(By.CLASS_NAME, 'o-form-button-bar') # Click Sign in
element.click()
time.sleep(5)

element = driver.find_element(By.ID, 'idBtn_Back')
element.click()
time.sleep(5)

In [ ]:
def get_name_from_email(email):
    element = driver.find_element(By.ID, 'topSearchInput')
    element.clear()
    element.send_keys(email+'@msu.edu')
    time.sleep(1)
    element = driver.find_element(By.CLASS_NAME, 'rPhnB') # To inspect on Mac, press Command + Option + C
    return element.text.replace('\n', '')

df['CMSE Faculty Name'] = df['CMSE Faculty NetID'].apply(lambda email: get_name_from_email(email))
df

In [ ]:
df.to_csv('CMSE_Names.csv', index=False)